HomeWork_5

Петренко, М3104

Задание 1

Примечание: jupyter notebook для задания 1 есть в репозитории по ссылке https://github.com/petmila/My_graphs_discrete_math

(a) Нарисовано с помощью сайта https://programforyou.ru/graph-redactor добавлением вершин и ребер вручную.

<img src="graph_europe.png"> 

Список вершин и их географических координат:

In [205]:
geolock = {"Armenia":{"height": 40,"width": 44},
           "Albania":{"height": 41,"width": 19},
           "Andorra":{"height": 42,"width": 1},
           "Austria":{"height": 48,"width": 16},
           "Belarus":{"height": 53,"width": 27},
           "Belgium":{"height": 50,"width": 4},
           "Bosnia_and_Herzegovina":{"height": 43,"width": 18},
           "Bulgaria":{"height": 42,"width": 23},
           "Croatia":{"height": 45,"width": 15},
           "Czech_Republic":{"height": 50,"width": 14},
           "Denmark":{"height": 55,"width": 12},
           "Estonia":{"height": 59,"width": 24},
           "Finland":{"height": 60,"width": 24},
           "France":{"height": 48,"width": 2},
           "Germany":{"height": 52,"width": 13},
           "Georgia":{"height": 41,"width": 44},
           "Greece":{"height": 37,"width": 23},
           "Hungary":{"height": 47,"width": 19},
           "Iceland":{"height": 64,"width": 21},
           "Ireland":{"height": 53,"width": 6},
           "Italy":{"height": 41,"width": 12},
           "Kosovo":{"height": 42,"width": 21},
           "Latvia":{"height": 56,"width": 24},
           "Liechtenstein":{"height": 47,"width": 9},
           "Lithuania":{"height": 54,"width": 25},
           "Luxembourg":{"height": 49,"width": 6},
           "Malta":{"height": 35,"width": 14},
           "Moldova":{"height": 47,"width": 28},
           "Monaco":{"height": 47,"width": 9},
           "Montenegro":{"height": 42,"width": 19},
           "Netherlands":{"height": 52,"width": 4},
           "North_Macedonia":{"height": 41,"width": 21},
           "Norway":{"height": 59,"width": 10},
           "Poland":{"height": 52,"width": 21},
           "Portugal":{"height": 38,"width": 9},
           "Romania":{"height": 47,"width": 19},
           "Russia":{"height": 55,"width": 37},
           "San_Marino":{"height": 43,"width": 12},
           "Serbia":{"height": 44,"width": 20},
           "Slovakia":{"height": 48,"width": 17},
           "Slovenia":{"height": 46,"width": 14},
           "Spain":{"height": 40,"width": 3},
           "Sweden":{"height": 59,"width": 18},
           "Switzerland":{"height": 46,"width": 7},
           "Turkey":{"height": 39,"width": 32},
           "Ukraine":{"height": 50,"width": 30},
           "United_Kingdom":{"height": 51,"width": 0},
           "Vatican_City":{"height": 41,"width": 12},
           }

In [206]:
import itertools
import matplotlib.pyplot as plt
import networkx as nx
import numpy.random as rnd
import haversine

class Europe:
    def __init__(self):
        self.graph = nx.read_edgelist("graph.txt")  # создание графа по списку ребер из файла(файл есть в репозитории)
        self.graph.add_node('Malta')
        self.graph.add_node('Iceland')  # добавляю островные государства
        self.graph.add_node('Cyprus')

        self.nodes_number = self.graph.number_of_nodes()
        self.edges_number = self.graph.number_of_edges()

        subgraphs = list(nx.connected_components(self.graph))
        self.connected_nodes_list = {}
        for subgraph in subgraphs:
            if len(subgraph) > len(self.connected_nodes_list):
                self.connected_nodes_list = subgraph
        self.largest_connected_component = nx.subgraph(self.graph, self.connected_nodes_list)

        self.degree = nx.degree(self.largest_connected_component)  # массив степеней всех вершин
        min_degree = self.nodes_number + 1
        max_degree = 0
        for node in self.degree:
            if node[1] < min_degree:
                min_degree = node[1]
            if node[1] > max_degree:
                max_degree = node[1]
        self.min_degree = min_degree
        self.max_degree = max_degree

        self.v_coloring = nx.greedy_color(self.largest_connected_component) # множество пар (вершина : номер цвета)
        
        self.dual = nx.line_graph(self.largest_connected_component) # создание дуального графа
        self.e_coloring = nx.greedy_color(self.dual)

        cliques = list(nx.find_cliques(self.largest_connected_component))
        self.max_clique = cliques[0]
        for clique in cliques:
            if len(self.max_clique) < len(clique):
                self.max_clique = clique
            
        self.max_stable_set = nx.maximal_independent_set(self.largest_connected_component)
        # для невзвешенного графа данная функция сочтет вес дефолтным - 1
        # так что она посчитает просто количество ребер
        self.max_matching = nx.max_weight_matching(self.largest_connected_component)
        self.v_cover = nx.approximation.min_weighted_vertex_cover(self.largest_connected_component)
        self.e_cover = nx.min_edge_cover(self.largest_connected_component)

        self.blocks = list(nx.biconnected_components(self.graph))


    def number_of_nodes(self):
        print("Number of nodes:", self.nodes_number)

    def number_of_edges(self):
        print("Number of edges:", self.edges_number)

    def degree(self):
        print("Minimum degree:", self.min_degree)
        print("Maximum degree:", self.max_degree)

    def diameter(self):
        print("Diameter:", nx.diameter(self.largest_connected_component))

    def center(self):
        print("Center:", nx.center(self.largest_connected_component))

    def radius(self):
        print("Radius:", nx.radius(self.largest_connected_component))

    def girth(self):
        subgraphs = list(nx.minimum_cycle_basis(self.largest_connected_component))
        nodes_list = self.nodes_number * [0]
        for subgraph in subgraphs:
            if len(subgraph) < len(nodes_list):
                nodes_list = subgraph
        print("Girth:", len(nodes_list))
    
    def vertex_coloring(self):
        print("Minimum vertex coloring:", max(self.v_coloring.values()))

    def edge_coloring(self):
        print("Minimum edge coloring:", max(self.e_coloring.values()))

    def maximum_clique(self):
        print("Maximum clique:", self.max_clique)

    def maximum_stable_set(self):
        print("Maximum stable set:", self.max_stable_set)

    def maximum_matching(self):
        print("Maximum matching:", self.max_matching)

    def minimum_vertex_cover(self):
        print("Minimum vertex cover:", self.v_cover)
    
    def minimum_edge_cover(self):
        print("Minimum edge cover:", self.e_cover)

    def all_blocks(self):
        print("Biconnected components:")
        for item in self.blocks:
            print(item)

    def all_2_edge_connected_components(self):
        print("2-edge-connected components:")
        for item in list(nx.biconnected_component_edges(self.graph)):
            print(item)

    def make_weighted_graph(self):
        self.weighted_graph = nx.read_edgelist("graph.txt")  # создание графа по списку ребер из файла(файл есть в репозитории)
        self.weighted_graph.add_node('Malta')
        self.weighted_graph.add_node('Iceland')  # добавляю островные государства
        self.weighted_graph.add_node('Cyprus')

        nx.set_node_attributes(self.weighted_graph, geolock)
        edges_weight = {}
        for item in nx.edges(self.weighted_graph):
            edges_weight[item] = int(haversine.haversine((self.weighted_graph.nodes[item[0]]["height"], self.weighted_graph.nodes[item[0]]["width"]),
            (self.weighted_graph.nodes[item[1]]["height"],self.weighted_graph.nodes[item[1]]["width"])))
        nx.set_edge_attributes(self.weighted_graph , edges_weight, 'weight')

        self.largest_weighted_connected_component = nx.subgraph(self.weighted_graph, self.connected_nodes_list)
    def spanning_tree(self):
        self.minimum_spanning_tree = nx.minimum_spanning_tree(self.largest_weighted_connected_component)
        print("Minimum_spanning_tree:")
        return self.minimum_spanning_tree
    

(b)

In [207]:
europe = Europe()
europe.number_of_edges()
europe.number_of_nodes()
europe.degree()
europe.diameter()
europe.center()
europe.radius()
europe.girth()


Number of edges: 92
Number of nodes: 49
Diameter: 8
Center: ['Poland']
Radius: 4
Girth: 3


(c)

In [208]:
europe.vertex_coloring()

Minimum vertex coloring: 4


(d)


In [209]:
europe.edge_coloring()

Minimum edge coloring: 8


(e)

In [210]:
europe.maximum_clique()

Maximum clique: ['Lithuania', 'Russia', 'Belarus', 'Poland']


(f)

In [211]:
europe.maximum_stable_set()

Maximum stable set: ['Denmark', 'Switzerland', 'Slovakia', 'Netherlands', 'Portugal', 'Norway', 'Estonia', 'Montenegro', 'Belarus', 'Turkey', 'Monaco', 'Luxembourg', 'Andorra', 'Slovenia', 'North_Macedonia', 'Romania', 'San_Marino', 'Vatican_City']


(g)

In [212]:
europe.maximum_matching()

Maximum matching: {('Denmark', 'Germany'), ('Austria', 'Slovenia'), ('Vatican_City', 'Italy'), ('Lithuania', 'Belarus'), ('Norway', 'Sweden'), ('Finland', 'Russia'), ('Estonia', 'Latvia'), ('Poland', 'Czech_Republic'), ('Montenegro', 'Bosnia_and_Herzegovina'), ('Turkey', 'Armenia'), ('Netherlands', 'Belgium'), ('Portugal', 'Spain'), ('Liechtenstein', 'Switzerland'), ('Croatia', 'Hungary'), ('Monaco', 'France'), ('Kosovo', 'Serbia'), ('Moldova', 'Romania'), ('Bulgaria', 'Greece'), ('Ukraine', 'Slovakia'), ('North_Macedonia', 'Albania')}


(h)

In [213]:
europe.minimum_vertex_cover()

Minimum vertex cover: {'Slovakia', 'Georgia', 'Austria', 'Andorra', 'Czech_Republic', 'Bosnia_and_Herzegovina', 'Romania', 'Germany', 'Russia', 'Hungary', 'Greece', 'Bulgaria', 'Kosovo', 'Belarus', 'Serbia', 'Croatia', 'Spain', 'Belgium', 'North_Macedonia', 'Turkey', 'Italy', 'Armenia', 'Finland', 'Albania', 'Latvia', 'France', 'Slovenia', 'Ukraine', 'Sweden', 'Switzerland', 'Poland'}


(i)

In [214]:
europe.minimum_edge_cover()

Minimum edge cover: {('Denmark', 'Germany'), ('Belgium', 'Luxembourg'), ('Austria', 'Slovenia'), ('Vatican_City', 'Italy'), ('Lithuania', 'Belarus'), ('Norway', 'Sweden'), ('Finland', 'Russia'), ('Estonia', 'Latvia'), ('Poland', 'Czech_Republic'), ('Andorra', 'Spain'), ('Montenegro', 'Bosnia_and_Herzegovina'), ('Turkey', 'Armenia'), ('Armenia', 'Georgia'), ('Netherlands', 'Belgium'), ('Portugal', 'Spain'), ('Luxembourg', 'Belgium'), ('Liechtenstein', 'Switzerland'), ('Italy', 'San_Marino'), ('Croatia', 'Hungary'), ('Monaco', 'France'), ('Kosovo', 'Serbia'), ('Spain', 'Andorra'), ('Moldova', 'Romania'), ('Bulgaria', 'Greece'), ('Ukraine', 'Slovakia'), ('Georgia', 'Armenia'), ('North_Macedonia', 'Albania'), ('San_Marino', 'Italy')}


(j)

(k)

(l)

In [215]:
europe.all_blocks()

Biconnected components:
{'Portugal', 'Spain'}
{'Andorra', 'France', 'Spain'}
{'San_Marino', 'Italy'}
{'Vatican_City', 'Italy'}
{'France', 'Monaco'}
{'Norway', 'Russia', 'Finland', 'Sweden'}
{'Germany', 'Denmark'}
{'Slovakia', 'Georgia', 'Austria', 'Czech_Republic', 'Lithuania', 'Bosnia_and_Herzegovina', 'Romania', 'Germany', 'Russia', 'Hungary', 'Greece', 'Bulgaria', 'Luxembourg', 'Belarus', 'Kosovo', 'Serbia', 'Croatia', 'Belgium', 'North_Macedonia', 'Turkey', 'Italy', 'Netherlands', 'Armenia', 'Montenegro', 'Albania', 'Latvia', 'France', 'Moldova', 'Liechtenstein', 'Slovenia', 'Ukraine', 'Switzerland', 'Poland', 'Estonia'}
{'Ireland', 'United_Kingdom'}


(m)

In [216]:
europe.all_2_edge_connected_components()

2-edge-connected components:
[('Spain', 'Portugal')]
[('France', 'Andorra'), ('Andorra', 'Spain'), ('Spain', 'France')]
[('Italy', 'San_Marino')]
[('Italy', 'Vatican_City')]
[('France', 'Monaco')]
[('Russia', 'Finland'), ('Finland', 'Sweden'), ('Sweden', 'Norway'), ('Norway', 'Finland'), ('Norway', 'Russia')]
[('Germany', 'Denmark')]
[('Armenia', 'Turkey'), ('Turkey', 'Bulgaria'), ('Bulgaria', 'Greece'), ('Greece', 'Albania'), ('Albania', 'Serbia'), ('Serbia', 'Bosnia_and_Herzegovina'), ('Bosnia_and_Herzegovina', 'Croatia'), ('Croatia', 'Hungary'), ('Hungary', 'Austria'), ('Austria', 'Germany'), ('Germany', 'Belgium'), ('Belgium', 'France'), ('France', 'Luxembourg'), ('Luxembourg', 'Belgium'), ('Luxembourg', 'Germany'), ('France', 'Germany'), ('France', 'Switzerland'), ('Switzerland', 'Austria'), ('Switzerland', 'Germany'), ('Switzerland', 'Italy'), ('Italy', 'Austria'), ('Italy', 'France'), ('Italy', 'Slovenia'), ('Slovenia', 'Austria'), ('Slovenia', 'Croatia'), ('Slovenia', 'Hungary'

(n)

(o)
<img src="spanning_tree.png"> 

In [218]:
europe.make_weighted_graph()
MST = europe.spanning_tree()
for i in MST.nodes():
    print(MST.edges(i))
print(MST)

Minimum_spanning_tree:
[('Armenia', 'Georgia'), ('Armenia', 'Turkey')]
[('Turkey', 'Greece'), ('Turkey', 'Armenia')]
[('Georgia', 'Armenia')]
[('Albania', 'North_Macedonia')]
[('Greece', 'North_Macedonia'), ('Greece', 'Turkey')]
[('Serbia', 'Bosnia_and_Herzegovina')]
[('Kosovo', 'North_Macedonia'), ('Kosovo', 'Montenegro')]
[('North_Macedonia', 'Kosovo'), ('North_Macedonia', 'Albania'), ('North_Macedonia', 'Bulgaria'), ('North_Macedonia', 'Greece')]
[('Andorra', 'Spain'), ('Andorra', 'France')]
[('Spain', 'Andorra'), ('Spain', 'Portugal')]
[('France', 'Belgium'), ('France', 'Switzerland'), ('France', 'Monaco'), ('France', 'Andorra')]
[('Austria', 'Slovakia'), ('Austria', 'Czech_Republic'), ('Austria', 'Slovenia'), ('Austria', 'Liechtenstein')]
[('Germany', 'Czech_Republic'), ('Germany', 'Denmark')]
[('Czech_Republic', 'Germany'), ('Czech_Republic', 'Austria')]
[('Slovakia', 'Austria'), ('Slovakia', 'Hungary'), ('Slovakia', 'Poland')]
[('Hungary', 'Romania'), ('Hungary', 'Slovakia')]
[(